# Isha Raju and Shreeya Rajanarayanan - Group 5 Project Code

# Summary

1. Setup
2. Convert files with downloaded tweets into JSON format
3. Connect to Postgres and create a Database
4. Connect to MongoDB and create a Database and a collection
5. Process Each tweet and assign the contents into
    1. USER information Database (POSTGRES)
    2. Tweet information Database (MongoDB)
   in one pass
6. Print Both POstgres(User information) and MongoDB(Tweet information) 
7. Apply indexing to MongoDB for faster querying
8. Search application with Cache


In [1]:
# pip install ipython-sql
# pip install sqlalchemy
# pip install psycopg2-binary
# pip uninstall psycopg2
# pip list --outdated
# from platform import python_version
# print(python_version())
# pip install --upgrade wheel
# pip install --upgrade pip
# pip install --upgrade setuptools
# pip install psycopg2
# pip install pymongo
# Brew services list
# brew services start postgresql
# brew services stop postgresql
# brew services start redis
# redis-cli ping
# sudo pip install redis
#pip install redis

In [2]:
#db.tweet_data.drop()
#db.tweet_data.drop()

# Setup

In [1]:
import os
import glob
import pandas as pd
import pymongo
import json
import psycopg2
import requests
import redis
from bson.json_util import dumps
import seaborn as sns
import matplotlib.pyplot as plt
import redis
%matplotlib inline
from datetime import datetime

# Convert files with downloaded tweets into JSON format

In [2]:
# function used to get JSON data from file originally used in MongoDB mini-project presentation
def get_json_data_from_file(source_filename):
    """
    Given a source file path of json format, this function returns the json data in the form of a python dictionary
    """
    with open(source_filename, "r") as read_file:
        data = json.load(read_file)
    return data

In [3]:
#tweets = get_json_data_from_file('out-covid3_comma_removed.json')
tweets = get_json_data_from_file('out-covid3.json')

In [4]:
#Check the first tweet
tweets[0]

{'created_at': 'Fri Apr 23 14:05:26 +0000 2021',
 'id': 1385595673479766024,
 'id_str': '1385595673479766024',
 'text': 'RT @LiveLawIndia: Court records - Dr. Somani, central drug controller has given vital info on remd. We appreciate his participation. Accord…',
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'truncated': False,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 235529423,
  'id_str': '235529423',
  'name': 'KUNWAR VIJAY RAGHAV SINGH',
  'screen_name': 'VIJAYraghavSING',
  'location': 'Jabalpur, India',
  'url': 'http://kunwarvijayraghavsingh.blogspot.com/',
  'description': 'ADVOCATE, HIGH COURT OF MADHYA PRADESH, PRINCIPAL SEAT AT JABALPUR (MADHYA PRADESH) +919826143925',
  'translator_type': 'none',
  'protected': False,
  'verified': False,
  'followers_count': 1029,
  'friends_count'

# RDBMS (Postgres Database ) consisting of User information

In [6]:
# create data table for user information in postgres
conn = psycopg2.connect("dbname=users_db port=5432 host=localhost ")
conn.set_session(autocommit = True)
cur = conn.cursor()

#cur.execute("DROP TABLE user_data;")

cur.execute("CREATE TABLE IF NOT EXISTS user_data ("   +
            "user_id              bigint PRIMARY KEY, "+
            "user_name            varchar(250), "      +
            "user_screen_name     varchar(250), "      +
            "user_verified        boolean, "           +
            "user_location        varchar(350), "      +
            "user_followers_count integer, "           +
            "user_friends_count   integer, "           +
            "user_favorites_count integer, "           +
            "user_status_count    integer, "           +
            "user_created_at      varchar(300));")
conn.commit()
cur.close()
conn.close()

In [7]:
# function to insert user data into postgres
def insert_user_data(user_id, user_name, user_screen_name, user_verified, user_location, user_followers_count, user_friends_count, user_favorites_count, user_status_count, user_created_at):
    conn   = psycopg2.connect("dbname='users_db' host='localhost' ")
    cur    = conn.cursor()
    insert = "INSERT INTO user_data (user_id, user_name, user_screen_name, user_verified, user_location, user_followers_count, user_friends_count, user_favorites_count, user_status_count, user_created_at) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s) ON CONFLICT (user_id) DO UPDATE SET user_name=EXCLUDED.user_name, user_screen_name=EXCLUDED.user_screen_name, user_verified=EXCLUDED.user_verified, user_location=EXCLUDED.user_location, user_followers_count=EXCLUDED.user_followers_count, user_friends_count=EXCLUDED.user_friends_count, user_favorites_count=EXCLUDED.user_favorites_count, user_status_count=EXCLUDED.user_status_count, user_created_at=EXCLUDED.user_created_at"
    cur.execute(insert, (user_id, user_name, user_screen_name, user_verified, user_location, user_followers_count, user_friends_count, user_favorites_count, user_status_count, user_created_at))
    conn.commit()
    cur.close()
    conn.close()
    

# NoSQL ( MongoDB Database) : tweet related information

Make sure MongoDB is running on your system using:
sudo mongod

In [8]:
from pymongo import MongoClient
client     = MongoClient('localhost', 27018)
db         = client['tweet_db']

#drop collections
db.tweet_data.drop()
db.tweet_data2.drop()

In [9]:
#Create database & collection for MongoDB 
#from pymongo import MongoClient
#client     = MongoClient('localhost', 27018)
#db         = client['tweet_db']

#create collection
tweet_data = db['tweet_data']

In [10]:
col_names  = db.list_collection_names()
print(col_names)

[]


In [11]:
# Use this fn to check if the data contains a specific object like retweeted_status
def check_for(data, obj):
    try:
        data[obj]
        return True
    except:
        return False

# Process Each tweet and assign the contents into 
# 1. USER information Database  (POSTGRES) 
# 2. Tweet information Database (MongoDB)
# in one pass


In [12]:
# read through tweets in one go and sort user data into postgres and tweet data into MongoDB
#keep count of number of tweets that were processed

import time
start_time = time.time()

tweet_counter = 0
for tweet in tweets:
    
    tweet_hashtags = []
    
    # if it's a regular tweet:
    if check_for(tweet, 'retweeted_status') == False:
        
        #_______________________________________________________________________________________________
        # gather info from regular tweet
        #_______________________________________________________________________________________________
        user_id              = tweet['user']['id']
        user_name            = tweet['user']['name']
        user_screen_name     = tweet['user']['screen_name']
        user_verified        = tweet['user']['verified']
        user_location        = tweet['user']['location']
        user_followers_count = tweet['user']['followers_count']
        user_friends_count   = tweet['user']['friends_count']
        user_favorites_count = tweet['user']['favourites_count']
        user_status_count    = tweet['user']['statuses_count']
        user_created_at      = tweet['user']['created_at']
       
                
        #_______________________________________________________________________________________________
        #gather tweet data to insert into MongoDB
        #_______________________________________________________________________________________________
        tweet_id             = tweet['id_str']
        tweet_favorite_count = tweet['favorite_count']
        tweet_retweet_count  = tweet['retweet_count']
        tweet_language       = tweet['lang']
        tweet_timestamp      = tweet['timestamp_ms']
        created_at           = tweet['created_at']
        tweet_created_at     = datetime.strftime(datetime.strptime(created_at,'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S')
        
        if check_for(tweet,'extended_tweet'):
            tweet_text = (tweet['extended_tweet']['full_text']).lower()
            for i in tweet['extended_tweet']['entities']['hashtags']:
                tweet_hashtags.append(i['text'].lower())
        else:
            tweet_text = (tweet['text']).lower()
            for i in tweet['entities']['hashtags']:
                tweet_hashtags.append(i['text'].lower())
                


        
    else: 
        
        #if its a retweet, only get data from original tweet
        #_______________________________________________________________________________________________
        #gather user info to insert into postgres
        #_______________________________________________________________________________________________
        user_id              = tweet['retweeted_status']['user']['id']
        user_name            = tweet['retweeted_status']['user']['name']
        user_screen_name     = tweet['retweeted_status']['user']['screen_name']
        user_verified        = tweet['retweeted_status']['user']['verified']
        user_location        = tweet['retweeted_status']['user']['location']
        user_followers_count = tweet['retweeted_status']['user']['followers_count']
        user_friends_count   = tweet['retweeted_status']['user']['friends_count']
        user_favorites_count = tweet['retweeted_status']['user']['favourites_count']
        user_status_count    = tweet['retweeted_status']['user']['statuses_count']
        user_created_at      = tweet['retweeted_status']['user']['created_at']
        #_______________________________________________________________________________________________
        #gather tweet data to insert into MongoDB
        #_______________________________________________________________________________________________
        tweet_id             = tweet['retweeted_status']['id_str']        
        tweet_favorite_count = tweet['retweeted_status']['favorite_count']
        tweet_retweet_count  = tweet['retweeted_status']['retweet_count']
        tweet_language       = tweet['retweeted_status']['lang']
        tweet_timestamp      = tweet['timestamp_ms']   #need to use timestamp from retweet because original timestamp unavailable: https://twittercommunity.com/t/timestamp-ms-key-missing-from-streaming-api-results/100029/3 
        created_at           = tweet['retweeted_status']['created_at']
        tweet_created_at     = datetime.strftime(datetime.strptime(created_at,'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S')

        
        if check_for(tweet['retweeted_status'],'extended_tweet'):
            tweet_text = (tweet['retweeted_status']['extended_tweet']['full_text']).lower()
            for i in tweet['retweeted_status']['extended_tweet']['entities']['hashtags']:
                tweet_hashtags.append(i['text'].lower())
        else:
            tweet_text = (tweet['retweeted_status']['text']).lower()
            for i in tweet['retweeted_status']['entities']['hashtags']:
                tweet_hashtags.append(i['text'].lower())
 

    #insert user data into postgres
    insert_user_data(user_id, user_name, user_screen_name, user_verified, user_location, user_followers_count, user_friends_count, user_favorites_count, user_status_count, user_created_at)
    


                    
    # Organize each row of tweet data            
    tweet_line = {'tweet_id'            : tweet_id,
                  'user_id'             : user_id,
                  'tweet_favorite_count': tweet_favorite_count,
                  'tweet_retweet_count' : tweet_retweet_count,
                  'tweet_language'      : tweet_language,
                  'tweet_timestamp'     : tweet_timestamp,
                  'tweet_text'          : tweet_text,
                  'tweet_hashtags'      : tweet_hashtags,
                  'tweet_created_at'    : tweet_created_at
                  }    
    
    
    #_______________________________________________________________________________________________
    #insert tweet data into MongoDB
    #_______________________________________________________________________________________________
    #check if tweet already in database
    
    if (tweet_data.count_documents({'tweet_id' : tweet_line['tweet_id']}) > 0) :db.tweet_data.update_one({'tweet_id' : tweet_line['tweet_id']},
                                    {'$set' : {'tweet_retweet_count'   : tweet_line['tweet_retweet_count'],
                                                'tweet_favorite_count'  : tweet_line['tweet_favorite_count'],
                                        }
                                       }
                                      )
    # if the tweet is new we insert it    
    else :
        tweet_data.insert_one(tweet_line)
        
    
    tweet_counter +=1
    print("processed_line ", tweet_counter)
        
        
print("--- %s seconds ---" % (time.time() - start_time))


processed_line  1
processed_line  2
processed_line  3
processed_line  4
processed_line  5
processed_line  6
processed_line  7
processed_line  8
processed_line  9
processed_line  10
processed_line  11
processed_line  12
processed_line  13
processed_line  14
processed_line  15
processed_line  16
processed_line  17
processed_line  18
processed_line  19
processed_line  20
processed_line  21
processed_line  22
processed_line  23
processed_line  24
processed_line  25
processed_line  26
processed_line  27
processed_line  28
processed_line  29
processed_line  30
processed_line  31
processed_line  32
processed_line  33
processed_line  34
processed_line  35
processed_line  36
processed_line  37
processed_line  38
processed_line  39
processed_line  40
processed_line  41
processed_line  42
processed_line  43
processed_line  44
processed_line  45
processed_line  46
processed_line  47
processed_line  48
processed_line  49
processed_line  50
processed_line  51
processed_line  52
processed_line  53
pr

processed_line  418
processed_line  419
processed_line  420
processed_line  421
processed_line  422
processed_line  423
processed_line  424
processed_line  425
processed_line  426
processed_line  427
processed_line  428
processed_line  429
processed_line  430
processed_line  431
processed_line  432
processed_line  433
processed_line  434
processed_line  435
processed_line  436
processed_line  437
processed_line  438
processed_line  439
processed_line  440
processed_line  441
processed_line  442
processed_line  443
processed_line  444
processed_line  445
processed_line  446
processed_line  447
processed_line  448
processed_line  449
processed_line  450
processed_line  451
processed_line  452
processed_line  453
processed_line  454
processed_line  455
processed_line  456
processed_line  457
processed_line  458
processed_line  459
processed_line  460
processed_line  461
processed_line  462
processed_line  463
processed_line  464
processed_line  465
processed_line  466
processed_line  467


processed_line  848
processed_line  849
processed_line  850
processed_line  851
processed_line  852
processed_line  853
processed_line  854
processed_line  855
processed_line  856
processed_line  857
processed_line  858
processed_line  859
processed_line  860
processed_line  861
processed_line  862
processed_line  863
processed_line  864
processed_line  865
processed_line  866
processed_line  867
processed_line  868
processed_line  869
processed_line  870
processed_line  871
processed_line  872
processed_line  873
processed_line  874
processed_line  875
processed_line  876
processed_line  877
processed_line  878
processed_line  879
processed_line  880
processed_line  881
processed_line  882
processed_line  883
processed_line  884
processed_line  885
processed_line  886
processed_line  887
processed_line  888
processed_line  889
processed_line  890
processed_line  891
processed_line  892
processed_line  893
processed_line  894
processed_line  895
processed_line  896
processed_line  897


processed_line  1256
processed_line  1257
processed_line  1258
processed_line  1259
processed_line  1260
processed_line  1261
processed_line  1262
processed_line  1263
processed_line  1264
processed_line  1265
processed_line  1266
processed_line  1267
processed_line  1268
processed_line  1269
processed_line  1270
processed_line  1271
processed_line  1272
processed_line  1273
processed_line  1274
processed_line  1275
processed_line  1276
processed_line  1277
processed_line  1278
processed_line  1279
processed_line  1280
processed_line  1281
processed_line  1282
processed_line  1283
processed_line  1284
processed_line  1285
processed_line  1286
processed_line  1287
processed_line  1288
processed_line  1289
processed_line  1290
processed_line  1291
processed_line  1292
processed_line  1293
processed_line  1294
processed_line  1295
processed_line  1296
processed_line  1297
processed_line  1298
processed_line  1299
processed_line  1300
processed_line  1301
processed_line  1302
processed_lin

processed_line  1649
processed_line  1650
processed_line  1651
processed_line  1652
processed_line  1653
processed_line  1654
processed_line  1655
processed_line  1656
processed_line  1657
processed_line  1658
processed_line  1659
processed_line  1660
processed_line  1661
processed_line  1662
processed_line  1663
processed_line  1664
processed_line  1665
processed_line  1666
processed_line  1667
processed_line  1668
processed_line  1669
processed_line  1670
processed_line  1671
processed_line  1672
processed_line  1673
processed_line  1674
processed_line  1675
processed_line  1676
processed_line  1677
processed_line  1678
processed_line  1679
processed_line  1680
processed_line  1681
processed_line  1682
processed_line  1683
processed_line  1684
processed_line  1685
processed_line  1686
processed_line  1687
processed_line  1688
processed_line  1689
processed_line  1690
processed_line  1691
processed_line  1692
processed_line  1693
processed_line  1694
processed_line  1695
processed_lin

...

processed_line  48129
processed_line  48130
processed_line  48131
processed_line  48132
processed_line  48133
processed_line  48134
processed_line  48135
processed_line  48136
processed_line  48137
processed_line  48138
processed_line  48139
processed_line  48140
processed_line  48141
processed_line  48142
processed_line  48143
processed_line  48144
processed_line  48145
processed_line  48146
processed_line  48147
processed_line  48148
processed_line  48149
processed_line  48150
processed_line  48151
processed_line  48152
processed_line  48153
processed_line  48154
processed_line  48155
processed_line  48156
processed_line  48157
processed_line  48158
processed_line  48159
processed_line  48160
processed_line  48161
processed_line  48162
processed_line  48163
processed_line  48164
processed_line  48165
processed_line  48166
processed_line  48167
processed_line  48168
processed_line  48169
processed_line  48170
processed_line  48171
processed_line  48172
processed_line  48173
processed_

processed_line  48507
processed_line  48508
processed_line  48509
processed_line  48510
processed_line  48511
processed_line  48512
processed_line  48513
processed_line  48514
processed_line  48515
processed_line  48516
processed_line  48517
processed_line  48518
processed_line  48519
processed_line  48520
processed_line  48521
processed_line  48522
processed_line  48523
processed_line  48524
processed_line  48525
processed_line  48526
processed_line  48527
processed_line  48528
processed_line  48529
processed_line  48530
processed_line  48531
processed_line  48532
processed_line  48533
processed_line  48534
processed_line  48535
processed_line  48536
processed_line  48537
processed_line  48538
processed_line  48539
processed_line  48540
processed_line  48541
processed_line  48542
processed_line  48543
processed_line  48544
processed_line  48545
processed_line  48546
processed_line  48547
processed_line  48548
processed_line  48549
processed_line  48550
processed_line  48551
processed_

processed_line  48882
processed_line  48883
processed_line  48884
processed_line  48885
processed_line  48886
processed_line  48887
processed_line  48888
processed_line  48889
processed_line  48890
processed_line  48891
processed_line  48892
processed_line  48893
processed_line  48894
processed_line  48895
processed_line  48896
processed_line  48897
processed_line  48898
processed_line  48899
processed_line  48900
processed_line  48901
processed_line  48902
processed_line  48903
processed_line  48904
processed_line  48905
processed_line  48906
processed_line  48907
processed_line  48908
processed_line  48909
processed_line  48910
processed_line  48911
processed_line  48912
processed_line  48913
processed_line  48914
processed_line  48915
processed_line  48916
processed_line  48917
processed_line  48918
processed_line  48919
processed_line  48920
processed_line  48921
processed_line  48922
processed_line  48923
processed_line  48924
processed_line  48925
processed_line  48926
processed_

processed_line  49255
processed_line  49256
processed_line  49257
processed_line  49258
processed_line  49259
processed_line  49260
processed_line  49261
processed_line  49262
processed_line  49263
processed_line  49264
processed_line  49265
processed_line  49266
processed_line  49267
processed_line  49268
processed_line  49269
processed_line  49270
processed_line  49271
processed_line  49272
processed_line  49273
processed_line  49274
processed_line  49275
processed_line  49276
processed_line  49277
processed_line  49278
processed_line  49279
processed_line  49280
processed_line  49281
processed_line  49282
processed_line  49283
processed_line  49284
processed_line  49285
processed_line  49286
processed_line  49287
processed_line  49288
processed_line  49289
processed_line  49290
processed_line  49291
processed_line  49292
processed_line  49293
processed_line  49294
processed_line  49295
processed_line  49296
processed_line  49297
processed_line  49298
processed_line  49299
processed_

processed_line  49632
processed_line  49633
processed_line  49634
processed_line  49635
processed_line  49636
processed_line  49637
processed_line  49638
processed_line  49639
processed_line  49640
processed_line  49641
processed_line  49642
processed_line  49643
processed_line  49644
processed_line  49645
processed_line  49646
processed_line  49647
processed_line  49648
processed_line  49649
processed_line  49650
processed_line  49651
processed_line  49652
processed_line  49653
processed_line  49654
processed_line  49655
processed_line  49656
processed_line  49657
processed_line  49658
processed_line  49659
processed_line  49660
processed_line  49661
processed_line  49662
processed_line  49663
processed_line  49664
processed_line  49665
processed_line  49666
processed_line  49667
processed_line  49668
processed_line  49669
processed_line  49670
processed_line  49671
processed_line  49672
processed_line  49673
processed_line  49674
processed_line  49675
processed_line  49676
processed_

In [13]:
conn = psycopg2.connect("dbname=users_db port=5432 host=localhost ")
cur = conn.cursor()


# take a PostgreSQL query and outputs a pandas database 
def create_pandas_table(sql_query, database) :
    table = pd.read_sql_query(sql_query, database)
    return table

In [14]:
#View postgres data in table
user_table = create_pandas_table("SELECT * FROM user_data",database = conn)
user_table

,user_id,user_name,user_screen_name,user_verified,user_location,user_followers_count,user_friends_count,user_favorites_count,user_status_count,user_created_at
0,894254235927298048,Savita Vantagodi,SavitaVantagodi,False,"Hubli, India",2541,2185,13806,45409,Sun Aug 06 17:49:59 +0000 2017
1,847141279,A Virtual College EST. 2010,pct2u,False,UK,1769,5001,1925,26185,Wed Sep 26 10:15:00 +0000 2012
2,252794509,André Picard,picardonhealth,True,None,122331,1884,1921,147530,Tue Feb 15 22:51:18 +0000 2011
3,2852217938,Lark Davis,DCryptoLark,False,"Wellington City, New Zealand",367,2185,49,399,Sat Oct 11 17:20:15 +0000 2014
4,3246541758,Medical Dialogues,medicaldialogs,False,"New Delhi, Delhi",5954,1946,47,37967,Tue Jun 16 07:34:34 +0000 2015
...,...,...,...,...,...,...,...,...,...,...
11277,1264374288603156481,MSG Fans UK,MSGFansUK1,False,None,1036,17,757,1019,Sun May 24 01:55:39 +0000 2020
11278,703017499,Dr. Mark Walker,DrMWalkerMD,False,Ottawa,882,193,2524,1976,Wed Jul 18 12:59:38 +0000 2012
11279,1188636039717376002,EmbaCuba China,EmbacubaChina,False,"Pekín, República Popular China",2520,1430,1415,1723,Mon Oct 28 01:58:18 +0000 2019
11280,3127085801,Anastasios Telloglou,telloglou,False,None,23104,2100,33684,101919,Sun Mar 29 18:17:58 +0000 2015


# Indexing

In [15]:
#add MongoDB index
tweet_data.create_index([ ('tweet_id', 1) ])
tweet_data.create_index([("tweet_text", pymongo.TEXT)])
tweet_data.create_index([ ('tweet_hashtags', 1) ])
tweet_data.create_index([ ('tweet_created_at', 1) ])

'tweet_created_at_1'

In [16]:
tweet_dt = pd.DataFrame(tweet_data.find())
tweet_dt

,_id,tweet_id,user_id,tweet_favorite_count,tweet_retweet_count,tweet_language,tweet_timestamp,tweet_text,tweet_hashtags,tweet_created_at
0,6092c80a648231edea74d521,1385565857493962756,1255161552,2,1,en,1619186726337,"court records - dr. somani, central drug contr...","[covid19, bombayhc, breakthechain, nagpur, rem...",2021-04-23 12:06:57
1,6092c80b648231edea74d522,1385577590887378944,891250296512323585,466,196,en,1619186726497,dedicated #covid19 beds in both govt and priva...,"[covid19, pmwithcms]",2021-04-23 12:53:35
2,6092c80b648231edea74d523,1385072861606387714,894254235927298048,95,83,en,1619186726470,“the victor will never be asked if he told the...,[],2021-04-22 03:27:58
3,6092c80b648231edea74d524,1385595673874157569,44961877,11,7,en,1619186726431,"today, @mdhealthdept reports 441,155 confirmed...",[covid19],2021-04-23 14:05:26
4,6092c80b648231edea74d525,1385546976356229123,245259319,152,168,en,1619186726603,"#oxygencrisis \nindian spinal injuries centre,...","[oxygencrisis, covid19]",2021-04-23 10:51:56
...,...,...,...,...,...,...,...,...,...,...
16165,6092cd54648231edea751446,1385543182600089601,1188636039717376002,3,2,en,1619194627777,all the municipalities of #havana will be part...,"[havana, soberana02, abdala, cubaporlasalud, c...",2021-04-23 10:36:51
16166,6092cd54648231edea751447,1385628815548944385,3127085801,0,0,en,1619194628022,with some delay vis a vis mhra,[],2021-04-23 16:17:08
16167,6092cd54648231edea751448,1384820456679567361,1274041003440103425,102,28,en,1619194628052,begin earning #pswap rewards at launch - our l...,"[pswap, tokenomics, cryptocurrency, crypto, de...",2021-04-21 10:45:00
16168,6092cd54648231edea751449,1379420159413207042,1095391940613414914,11,6,en,1619194628258,"thank you, to the amazing $zil community for u...","[cryptocurrenices, zilliqa, zilliqacommunity, ...",2021-04-06 13:06:09


# Search application 

In [28]:
# Search tweets by a word
def search_word():
    query = {"$text":{"$search": user_input}}
    twts = pd.DataFrame(tweet_data.find(query)).reset_index()
    return twts

In [29]:
# Search tweets with a hashtag
def search_hashtag():
    query = {"tweet_hashtags": { "$in" : [user_input]}}
    twts = pd.DataFrame(tweet_data.find(query)).reset_index()
    return twts

In [30]:
# Search tweets within a datetime range
def search_date():
    twts = pd.DataFrame(tweet_data.find({"tweet_created_at":{ "$gte": start_date, "$lt": end_date }})).reset_index()
    return twts

In [31]:
# Searchtweets with the tweet_id
def search_tweet_id():
    query = {'tweet_id': user_input}
    twts = pd.DataFrame(tweet_data.find(query)).reset_index()
    return twts

In [32]:
def return_myquery_key():
    user = input("Please enter whether you want user information or tweet information?: \n-> For user information enter 1 \n-> For tweet information enter 0\n" )

    if user == "1":
        filter_ = input("What information would you like to see: \n->  For number of friends of users enter 1 \n->  For number of followers of users enter 2  \n->  For location of users enter 3\n" )
        no_records = input("By default 5 records will be displayed by descending order of the previous selection. How many records do you want to display\n-> Enter the number of your choice \n-> else enter 5 \n")
        myquery = user + filter_
        myquery1 = myquery
        return user, myquery, no_records
    else :
        by_ = input("To search by \n->  Hashtag enter 1 \n->  Keyword enter 2  \n->  Timeframe enter 3\n->  Tweet_id enter 4\n" )
        no_records0 = input("By default 5 records will be displayed. How many records do you want to display\n-> Enter the number of your choice \n-> else enter 5 \n")
        myquery = user + by_
        
        if myquery =="01":
                user_input = input("Please enter a word: ")   
                myquery1 = myquery + user_input
                return myquery, myquery1, user, user_input, no_records0
        elif myquery =="02":
                user_input = input("Please enter a hashtag(exclude #) in lowcase: ")
                myquery1 = myquery + user_input
                return myquery, myquery1, user, user_input, no_records0
        elif myquery =="03":
                start_date = input("Please enter a start date(format:yyyy-mm-dd hh:mm:ss): ")
                end_date = input("Please enter a end date(format:yyyy-mm-dd hh:mm:ss): ")
                myquery1 = myquery + start_date + end_date
                return myquery, myquery1, user, start_date, end_date, no_records0 
        elif myquery =="04":
                user_input = input("Please enter a tweet id: ")
                myquery1 = myquery + user_input
                return myquery, myquery1, user, user_input, no_records0

        

In [33]:
def search_queries(myquery):

    if myquery =="11":
        cur.execute("select user_id, user_name, user_friends_count from user_data order by user_friends_count desc limit " + no_records)
        top_count = pd.DataFrame(cur.fetchall())
        top_count.columns = [desc[0] for desc in cur.description]
        return top_count

    elif myquery =="12":
        cur.execute("select user_id, user_name, user_followers_count from user_data order by user_followers_count desc limit " + no_records)
        top_count = pd.DataFrame(cur.fetchall())
        top_count.columns = [desc[0] for desc in cur.description]
        return top_count

    elif myquery =="13":
        cur.execute("select user_id, user_name, user_location from user_data where user_location = 'Europe' limit " + no_records)
        top_count = pd.DataFrame(cur.fetchall())
        top_count.columns = [desc[0] for desc in cur.description]
        return top_count

    elif myquery =="01":
        top_count =search_hashtag()
        return top_count.iloc[0:int(no_records0), :]
    
    elif myquery =="02":
        top_count = search_word()
        return top_count.iloc[0:int(no_records0), :]
    
    elif myquery =="03":
        top_count =search_date()
        return top_count.iloc[0:int(no_records0), :]
    
    elif myquery =="04":
        top_count =search_tweet_id()
        return top_count.iloc[0:int(no_records0), :]
            


# Caching

In [54]:
def caching(user, myquery1):
    # Connect to the Redis server 
    cache = redis.StrictRedis(connection_pool = globals()['cache_pool'])  

    if cache.exists(myquery1):
        print('Time taken when retrieving from Cache') 

        results = pd.read_json(cache.get(myquery1))
        print(results)

    else:
        print('Time taken when not in Cache')
        
        if user == "1": 
            top_count = search_queries(myquery1)
        else:
            top_count = search_queries(myquery)
            
            
        db      = client["tweet_database"]
        tweets  = db.tweets_collection 

        # Save in cache
        serializedObj = dumps(top_count)
        cache.set(myquery1, serializedObj)

        # keep for 60 seconds 
        cache.expire(myquery1, (60)) 

        results = top_count

        print(results)

# Performance

In [24]:
client = MongoClient('localhost', 27018)
globals()['client'] = client
cache_pool = redis.ConnectionPool(host = 'localhost', port = 6379, db = 0, max_connections = 2)
globals()['cache_pool'] = cache_pool

In [25]:
client.test_database

Database(MongoClient(host=['localhost:27018'], document_class=dict, tz_aware=False, connect=True), 'test_database')

# User search

## Time taken to search number of friends of a user with & without Caching

In [36]:
import time
user, myquery_friend, no_records = return_myquery_key()  
start_time = time.time()
caching(user, myquery_friend)
print("--- %s seconds ---" % (time.time() - start_time))

Please enter whether you want user information or tweet information?: 
-> For user information enter 1 
-> For tweet information enter 0
1
What information would you like to see: 
->  For number of friends of users enter 1 
->  For number of followers of users enter 2  
->  For location of users enter 3
1
By default 5 records will be displayed by descending order of the previous selection. How many records do you want to display
-> Enter the number of your choice 
-> else enter 5 
3
Time taken when not in Cache
     user_id                user_name  user_friends_count
0   15210670  Harjinder Singh Kukreja             1436314
1  200583835             Tim Fargo 😷💉              488112
2   35203319      🟣 Evan Kirstel $B2B              275322
--- 0.319537878036499 seconds ---


In [37]:

user, myquery_friend1, no_records = return_myquery_key()  
start_time = time.time()
caching(user, myquery_friend1)
print("--- %s seconds ---" % (time.time() - start_time))

Please enter whether you want user information or tweet information?: 
-> For user information enter 1 
-> For tweet information enter 0
1
What information would you like to see: 
->  For number of friends of users enter 1 
->  For number of followers of users enter 2  
->  For location of users enter 3
1
By default 5 records will be displayed by descending order of the previous selection. How many records do you want to display
-> Enter the number of your choice 
-> else enter 5 
3
Time taken when retrieving from Cache
     user_id                user_name  user_friends_count
0   15210670  Harjinder Singh Kukreja             1436314
1  200583835             Tim Fargo 😷💉              488112
2   35203319      🟣 Evan Kirstel $B2B              275322
--- 0.13479995727539062 seconds ---


## Time taken to search number of followers of a user with & without Caching

In [38]:
import time
user, myquery_follower, no_records = return_myquery_key()  
start_time = time.time()
caching(user, myquery_follower)
print("--- %s seconds ---" % (time.time() - start_time))

Please enter whether you want user information or tweet information?: 
-> For user information enter 1 
-> For tweet information enter 0
1
What information would you like to see: 
->  For number of friends of users enter 1 
->  For number of followers of users enter 2  
->  For location of users enter 3
2
By default 5 records will be displayed by descending order of the previous selection. How many records do you want to display
-> Enter the number of your choice 
-> else enter 5 
3
Time taken when not in Cache
    user_id       user_name  user_followers_count
0  18839785   Narendra Modi              67334826
1  37034483            NDTV              15052866
2  14159148  United Nations              13848616
--- 0.032950639724731445 seconds ---


In [40]:
import time
user, myquery_follower1, no_records = return_myquery_key()  
start_time = time.time()
caching(user, myquery_follower1)
print("--- %s seconds ---" % (time.time() - start_time))

Please enter whether you want user information or tweet information?: 
-> For user information enter 1 
-> For tweet information enter 0
1
What information would you like to see: 
->  For number of friends of users enter 1 
->  For number of followers of users enter 2  
->  For location of users enter 3
2
By default 5 records will be displayed by descending order of the previous selection. How many records do you want to display
-> Enter the number of your choice 
-> else enter 5 
3
Time taken when retrieving from Cache
    user_id       user_name  user_followers_count
0  18839785   Narendra Modi              67334826
1  37034483            NDTV              15052866
2  14159148  United Nations              13848616
--- 0.014451742172241211 seconds ---


## Time taken to search users by location with & without Caching

In [41]:
import time
user, myquery_loc, no_records = return_myquery_key()  
start_time = time.time()
caching(user, myquery_loc)
print("--- %s seconds ---" % (time.time() - start_time))

Please enter whether you want user information or tweet information?: 
-> For user information enter 1 
-> For tweet information enter 0
1
What information would you like to see: 
->  For number of friends of users enter 1 
->  For number of followers of users enter 2  
->  For location of users enter 3
3
By default 5 records will be displayed by descending order of the previous selection. How many records do you want to display
-> Enter the number of your choice 
-> else enter 5 
3
Time taken when not in Cache
               user_id      user_name user_location
0  1290605502834135041  Crypto Dizzle        Europe
1  1362836734304800770      GRVIPER99        Europe
2            156776475   Beatriz Ríos        Europe
--- 0.12944889068603516 seconds ---


In [42]:
import time
user, myquery_loc1, no_records = return_myquery_key()  
start_time = time.time()
caching(user, myquery_loc1)
print("--- %s seconds ---" % (time.time() - start_time))

Please enter whether you want user information or tweet information?: 
-> For user information enter 1 
-> For tweet information enter 0
1
What information would you like to see: 
->  For number of friends of users enter 1 
->  For number of followers of users enter 2  
->  For location of users enter 3
3
By default 5 records will be displayed by descending order of the previous selection. How many records do you want to display
-> Enter the number of your choice 
-> else enter 5 
3
Time taken when retrieving from Cache
               user_id      user_name user_location
0  1290605502834135041  Crypto Dizzle        Europe
1  1362836734304800770      GRVIPER99        Europe
2            156776475   Beatriz Ríos        Europe
--- 0.012928962707519531 seconds ---


# Tweet search

## Time taken to search a keyword with & without Caching

In [55]:
import time
myquery, myquery1, user, user_input, no_records0 = return_myquery_key()
start_time = time.time()
caching(user, myquery1)
print("--- %s seconds ---" % (time.time() - start_time))

Please enter whether you want user information or tweet information?: 
-> For user information enter 1 
-> For tweet information enter 0
0
To search by 
->  Hashtag enter 1 
->  Keyword enter 2  
->  Timeframe enter 3
->  Tweet_id enter 4
1
By default 5 records will be displayed. How many records do you want to display
-> Enter the number of your choice 
-> else enter 5 
3
Please enter a word: mask
Time taken when not in Cache
   index                       _id             tweet_id              user_id  \
0      0  6092c819648231edea74d854  1385591796747575300           2988288404   
1      1  6092c843648231edea74dd58  1385598453049352194  1293906975227404290   
2      2  6092c853648231edea74df85  1385599339544944647           4001651412   

   tweet_favorite_count  tweet_retweet_count tweet_language tweet_timestamp  \
0                     0                    2             en   1619186936448   
1                     0                    0             en   1619187389038   
2          

In [56]:
myquery, myquery1, user, user_input, no_records0 = return_myquery_key()  
start_time = time.time()
caching(user, myquery1)
print("--- %s seconds ---" % (time.time() - start_time))

Please enter whether you want user information or tweet information?: 
-> For user information enter 1 
-> For tweet information enter 0
0
To search by 
->  Hashtag enter 1 
->  Keyword enter 2  
->  Timeframe enter 3
->  Tweet_id enter 4
1
By default 5 records will be displayed. How many records do you want to display
-> Enter the number of your choice 
-> else enter 5 
3
Please enter a word: mask
Time taken when retrieving from Cache
   index                                   _id             tweet_id  \
0      0  {'$oid': '6092c819648231edea74d854'}  1385591796747575296   
1      1  {'$oid': '6092c843648231edea74dd58'}  1385598453049352192   
2      2  {'$oid': '6092c853648231edea74df85'}  1385599339544944640   

               user_id  tweet_favorite_count  tweet_retweet_count  \
0           2988288404                     0                    2   
1  1293906975227404290                     0                    0   
2           4001651412                     0                    0   

## Time taken to search a hashtag with & without Caching

In [57]:
myquery, myquery1, user, user_input, no_records0 = return_myquery_key()  
start_time = time.time()
caching(user, myquery1)
print("--- %s seconds ---" % (time.time() - start_time))

Please enter whether you want user information or tweet information?: 
-> For user information enter 1 
-> For tweet information enter 0
0
To search by 
->  Hashtag enter 1 
->  Keyword enter 2  
->  Timeframe enter 3
->  Tweet_id enter 4
2
By default 5 records will be displayed. How many records do you want to display
-> Enter the number of your choice 
-> else enter 5 
3
Please enter a hashtag(exclude #) in lowcase: trump
Time taken when not in Cache
   index                       _id             tweet_id              user_id  \
0      0  6092c8a4648231edea74e4a2  1385589133637156874            172748155   
1      1  6092c9e5648231edea74f87a  1385611817628536850  1256824219532316674   
2      2  6092c80c648231edea74d55b  1385594367788867590             22027975   

   tweet_favorite_count  tweet_retweet_count tweet_language tweet_timestamp  \
0                     3                    2             en   1619188169702   
1                     0                    0             en   16

In [60]:
myquery, myquery1, user, user_input, no_records0 = return_myquery_key()  
start_time = time.time()
caching(user, myquery1)
print("--- %s seconds ---" % (time.time() - start_time))

Please enter whether you want user information or tweet information?: 
-> For user information enter 1 
-> For tweet information enter 0
0
To search by 
->  Hashtag enter 1 
->  Keyword enter 2  
->  Timeframe enter 3
->  Tweet_id enter 4
2
By default 5 records will be displayed. How many records do you want to display
-> Enter the number of your choice 
-> else enter 5 
3
Please enter a hashtag(exclude #) in lowcase: trump
Time taken when retrieving from Cache
   index                                   _id             tweet_id  \
0      0  {'$oid': '6092c8a4648231edea74e4a2'}  1385589133637156864   
1      1  {'$oid': '6092c9e5648231edea74f87a'}  1385611817628536832   
2      2  {'$oid': '6092c80c648231edea74d55b'}  1385594367788867584   

               user_id  tweet_favorite_count  tweet_retweet_count  \
0            172748155                     3                    2   
1  1256824219532316674                     0                    0   
2             22027975                  10

## Time taken to search in a particular timeframe

In [63]:
import time
myquery, myquery1, user, start_date, end_date, no_records0 = return_myquery_key()  
start_time = time.time()
caching(user, myquery1)
print("--- %s seconds ---" % (time.time() - start_time))

Please enter whether you want user information or tweet information?: 
-> For user information enter 1 
-> For tweet information enter 0
0
To search by 
->  Hashtag enter 1 
->  Keyword enter 2  
->  Timeframe enter 3
->  Tweet_id enter 4
3
By default 5 records will be displayed. How many records do you want to display
-> Enter the number of your choice 
-> else enter 5 
3
Please enter a start date(format:yyyy-mm-dd hh:mm:ss): 2021-03-01
Please enter a end date(format:yyyy-mm-dd hh:mm:ss): 2021-05-01
Time taken when not in Cache
   index                       _id             tweet_id              user_id  \
0      0  6092c988648231edea74f4a4  1366210500204314628             94637037   
1      1  6092cb00648231edea750371  1366281411989233666  1183698123719200768   
2      2  6092cb78648231edea75076d  1366301397541552134           4316815947   

   tweet_favorite_count  tweet_retweet_count tweet_language tweet_timestamp  \
0                    76                   23             en   161

In [64]:
myquery, myquery1, user, start_date, end_date, no_records0 = return_myquery_key()  
start_time = time.time()
caching(user, myquery1)
print("--- %s seconds ---" % (time.time() - start_time))

Please enter whether you want user information or tweet information?: 
-> For user information enter 1 
-> For tweet information enter 0
0
To search by 
->  Hashtag enter 1 
->  Keyword enter 2  
->  Timeframe enter 3
->  Tweet_id enter 4
3
By default 5 records will be displayed. How many records do you want to display
-> Enter the number of your choice 
-> else enter 5 
3
Please enter a start date(format:yyyy-mm-dd hh:mm:ss): 2021-03-01
Please enter a end date(format:yyyy-mm-dd hh:mm:ss): 2021-05-01
Time taken when retrieving from Cache
   index                                   _id             tweet_id  \
0      0  {'$oid': '6092c988648231edea74f4a4'}  1366210500204314624   
1      1  {'$oid': '6092cb00648231edea750371'}  1366281411989233664   
2      2  {'$oid': '6092cb78648231edea75076d'}  1366301397541552128   

               user_id  tweet_favorite_count  tweet_retweet_count  \
0             94637037                    76                   23   
1  1183698123719200768           

## Time taken to search a tweet_id with or without caching

In [66]:

myquery, myquery1, user, user_input, no_records0 = return_myquery_key()  
start_time = time.time()
caching(user, myquery1)
print("--- %s seconds ---" % (time.time() - start_time))

Please enter whether you want user information or tweet information?: 
-> For user information enter 1 
-> For tweet information enter 0
0
To search by 
->  Hashtag enter 1 
->  Keyword enter 2  
->  Timeframe enter 3
->  Tweet_id enter 4
4
By default 5 records will be displayed. How many records do you want to display
-> Enter the number of your choice 
-> else enter 5 
1
Please enter a tweet id: 1366301397541552134
Time taken when not in Cache
   index                       _id             tweet_id     user_id  \
0      0  6092cb78648231edea75076d  1366301397541552134  4316815947   

   tweet_favorite_count  tweet_retweet_count tweet_language tweet_timestamp  \
0                    19                   22             en   1619192780945   

                                          tweet_text  \
0  in support of putting billions of human lives ...   

                              tweet_hashtags     tweet_created_at  
0  [tripswaiver, covid19, nocovidmonopolies]  2021-03-01 08:16:52  

In [71]:

myquery, myquery1, user, user_input, no_records0 = return_myquery_key()  
start_time = time.time()
caching(user, myquery1)
print("--- %s seconds ---" % (time.time() - start_time))

Please enter whether you want user information or tweet information?: 
-> For user information enter 1 
-> For tweet information enter 0
0
To search by 
->  Hashtag enter 1 
->  Keyword enter 2  
->  Timeframe enter 3
->  Tweet_id enter 4
4
By default 5 records will be displayed. How many records do you want to display
-> Enter the number of your choice 
-> else enter 5 
1
Please enter a tweet id: 1366301397541552134
Time taken when retrieving from Cache
   index                                   _id             tweet_id  \
0      0  {'$oid': '6092cb78648231edea75076d'}  1366301397541552128   

      user_id  tweet_favorite_count  tweet_retweet_count tweet_language  \
0  4316815947                    19                   22             en   

   tweet_timestamp                                         tweet_text  \
0    1619192780945  in support of putting billions of human lives ...   

                              tweet_hashtags    tweet_created_at  
0  [tripswaiver, covid19, nocovid